## 形参设定

In [1]:
import argparse
parser = argparse.ArgumentParser("mini-batch HAN")
parser.add_argument("-s", "--seed", type=int, default=1, help="Random seed")
parser.add_argument("--batch_size", type=int, default=32)
parser.add_argument("--num_neighbors", type=int, default=20)
parser.add_argument("--lr", type=float, default=0.001)
parser.add_argument("--num_heads", type=list, default=[8])
parser.add_argument("--hidden_units", type=int, default=8)
parser.add_argument("--dropout", type=float, default=0.6)
parser.add_argument("--weight_decay", type=float, default=0.001)
parser.add_argument("--num_epochs", type=int, default=100)
parser.add_argument("--patience", type=int, default=10)
parser.add_argument("--dataset", type=str, default="ACMRaw")
parser.add_argument("--device", type=str, default="cpu")

args, _ = parser.parse_known_args()

## 一、数据预处理

## 获取acm数据集

In [2]:
from dgl.data.utils import _get_dgl_url, download, get_download_dir
import dgl
import numpy as np
import torch
import pickle
from scipy import io as sio, sparse

## 设定acm数据集下载路径
url = "dataset/ACM.mat"
data_path = get_download_dir() + "/ACM.mat"
download(_get_dgl_url(url), path=data_path)


'C:\\Users\\Wei Zhou\\.dgl/ACM.mat'

## 提取相关的矩阵以下有四种矩阵

In [3]:
data = sio.loadmat(data_path)
p_vs_l = data["PvsL"]  # paper-field
p_vs_a = data["PvsA"]  # paper-author
p_vs_t = data["PvsT"]  # paper-term, bag of words
p_vs_c = data["PvsC"]  # paper-conference, labels come from that

print(f'一共有{p_vs_l.shape[0]}篇paper，一共有{p_vs_l.shape[1]}个领域')
print(f'一共有{p_vs_a.shape[0]}篇paper，一共有{p_vs_a.shape[1]}个作者')
print(f'一共有{p_vs_t.shape[0]}篇paper，一共有{p_vs_t.shape[1]}个专有名词')
print(f'一共有{p_vs_c.shape[0]}篇paper，一共有{p_vs_c.shape[1]}个会议期刊')

一共有12499篇paper，一共有73个领域
一共有12499篇paper，一共有17431个作者
一共有12499篇paper，一共有1903个专有名词
一共有12499篇paper，一共有14个会议期刊


## 根据p_vs_c矩阵选取我们需要的相关领域的期刊会议的paper

In [4]:
# We assign
# (1) KDD papers as class 0 (data mining),
# (2) SIGMOD and VLDB papers as class 1 (database),
# (3) SIGCOMM and MOBICOMM papers as class 2 (communication)
## 确定选取的期刊会议
conf_ids = [0, 1, 9, 10, 13]
## 对应期刊的领域的标签进行设定0为数据挖掘领域，1为数据库领域，2为通信领域
label_ids = [0, 1, 2, 2, 1]

In [5]:
##选取五个会议期刊
p_vs_c_filter = p_vs_c[:, conf_ids]

##将矩阵按照行求和，然后设定其不等于0从而提取和这五个期刊会议相关的paper，再获取这些paper的编码作为我们所需要的paper
p_selected = (p_vs_c_filter.sum(1) != 0).A1.nonzero()[0]

## 得到特定的paper后对所有的paper_vs的矩阵取出我们所需要的paper的信息
p_vs_l = p_vs_l[p_selected]
p_vs_a = p_vs_a[p_selected]
p_vs_t = p_vs_t[p_selected]
p_vs_c = p_vs_c[p_selected]

## 利用p_vs_a，p_vs_l构建异构图元路径

In [6]:
hg = dgl.heterograph(
    {
        ("paper", "pa", "author"): p_vs_a.nonzero(),
        ("author", "ap", "paper"): p_vs_a.transpose().nonzero(),
        ("paper", "pf", "field"): p_vs_l.nonzero(),
        ("field", "fp", "paper"): p_vs_l.transpose().nonzero(),
    }
)

# 利用p_vs_t构建节点特征矩阵

In [7]:
features_adj = torch.FloatTensor(p_vs_t.toarray())

## 对被选取的paper进行重新编码

In [8]:
pc_p, pc_c = p_vs_c.nonzero()
label = np.zeros(len(p_selected), dtype=np.int64)
for conf_id, label_id in zip(conf_ids, label_ids):
    label[pc_p[pc_c == conf_id]] = label_id
label = torch.LongTensor(label)

## 利用mask来决定哪些paper作为训练集，测试集，验证集

In [9]:
##随机生成的浮点数值，根据这些值将论文分配到不同的数据集中
num_classes = 3
float_mask = np.zeros(len(pc_p))
for conf_id in conf_ids:
    pc_c_mask = pc_c == conf_id
    float_mask[pc_c_mask] = np.random.permutation(
        np.linspace(0, 1, pc_c_mask.sum())
    )
train_idx = np.where(float_mask <= 0.2)[0]
val_idx = np.where((float_mask > 0.2) & (float_mask <= 0.3))[0]
test_idx = np.where(float_mask > 0.3)[0]

In [10]:
def get_binary_mask(total_size, indices):
    mask = torch.zeros(total_size)
    mask[indices] = 1
    return mask.byte()

In [11]:
num_nodes = hg.num_nodes("paper")
train_masks = get_binary_mask(num_nodes, train_idx)
val_masks = get_binary_mask(num_nodes, val_idx)
test_masks = get_binary_mask(num_nodes, test_idx)

## 该数据预处理的结果

In [12]:
## 由paper和author，paper和filed的连接关系组成的异构图
g=hg
## 由paper所包含的学术词汇构成的节点特征矩阵
features=features_adj
## 节点类型即每个paper属于哪个领域
labels=label
## 分类数量
n_classes=num_classes
## 训练集的节点索引
train_nid=train_idx
## 验证集节点索引
val_nid=val_idx
## 测试集节点索引
test_nid=test_idx
## 索引对应的掩码
train_mask=train_masks
val_mask=val_masks
test_mask=test_masks
## 元路径list
metapath_list = [["pa", "ap"], ["pf", "fp"]]

## 构建HAN采样——HANSampler 类的目标是为了构建用于训练的局部子图块，

In [13]:
from dgl.sampling import RandomWalkNeighborSampler
class HANSampler(object):
    def __init__(self, g, metapath_list, num_neighbors):
        self.sampler_list = []
##遍历元路径列表中的每个路径，并对每个元路径进行随机采样并将采样结果加入到sampler_list中
        for metapath in metapath_list:
            # note: random walk may get same route(same edge), which will be removed in the sampled graph.
            # So the sampled graph's edges may be less than num_random_walks(num_neighbors).
            self.sampler_list.append(
                RandomWalkNeighborSampler(
                ##在哪个图上进行采样
                    G=g,
                ##每个采样器执行随机游走的次数
                    num_traversals=1,
                ## 控制随机游走的终止概率
                    termination_prob=0,
                ##每个节点执行几次随机游走
                    num_random_walks=num_neighbors,
                ##每个节点采样的邻居节点的数量
                    num_neighbors=num_neighbors,
                ##使用的元路径是哪个
                    metapath=metapath,
                )
            )
            
## 构建训练子图
    def sample_blocks(self, seeds):
        block_list = []
##遍历采样列表
        for sampler in self.sampler_list:
        ##得到子图
            frontier = sampler(seeds)
        ##移除环
            frontier = dgl.remove_self_loop(frontier)
        ##添加自连接的边
            frontier.add_edges(torch.tensor(seeds), torch.tensor(seeds))
        ##将子图转化为block
            block = dgl.to_block(frontier, seeds)
            block_list.append(block)

        return seeds, block_list

In [14]:
num_neighbors = args.num_neighbors
han_sampler = HANSampler(g, metapath_list, num_neighbors)

In [15]:
## 提取子图block的节点特征
def load_subtensors(blocks, features):
    h_list = []
    for block in blocks:
        input_nodes = block.srcdata[dgl.NID]
        h_list.append(features[input_nodes])
    return h_list

## 二、定义加载器

In [16]:
from torch.utils.data import DataLoader

In [17]:
dataloader = DataLoader(
        dataset=train_nid,
        batch_size=args.batch_size,
##处理数据块（subgraph）的采样和组装。
        collate_fn=han_sampler.sample_blocks,
        shuffle=True,
        drop_last=False,
        num_workers=0,
##参数表示是否将加载的数据存储到CUDA固定内存中。
        pin_memory=False
    )

## 三、HAN模型设定

In [18]:
from dgl.nn.pytorch import GATConv
import torch.nn as nn
import torch.nn.functional as F

class SemanticAttention(nn.Module):
    def __init__(self, in_size, hidden_size=128):
        super(SemanticAttention, self).__init__()
        self.project = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False),
        )

    def forward(self, z):
    ##元路径在异构图中的重要性
        w = self.project(z).mean(0)  # (M, 1)
    ##元路径在异构图的注意力得分
        beta = torch.softmax(w, dim=0)  # (M, 1)
    ##对元路径注意力得分矩阵进行扩张，扩张为第一维度——节点数量，以便于最后的node_embedding生成的最终表示
        beta = beta.expand((z.shape[0],) + beta.shape)  # (N, M, 1)
    ##根据元路径的注意力得分矩阵和节点语义embedding生成最终的节点embedding
        return (beta * z).sum(1)  # (N, D * K)

## HAN layer构建

In [19]:
class HANLayer(torch.nn.Module):
    def __init__(
        self, num_metapath, in_size, out_size, layer_num_heads, dropout
    ):
        super(HANLayer, self).__init__()

        # One GAT layer for each meta path based adjacency matrix
        ## 节点级别的注意力机制 
        #构建 type-specific transformation matrix 特征变化矩阵
        self.gat_layers = nn.ModuleList()
        for i in range(num_metapath):
            self.gat_layers.append(
                GATConv(
                    in_size,
                    out_size,
                    layer_num_heads,
                    dropout,
                    dropout,
                    activation=F.elu,
                    allow_zero_in_degree=True,
                )
            )
        ## 语义级别的注意力机制
        self.semantic_attention = SemanticAttention(
            in_size=out_size * layer_num_heads
        )
        self.num_metapath = num_metapath

    def forward(self, block_list, h_list):
        semantic_embeddings = []
## 遍历每个元路径的子图列表并且获得基于该元路径的节点的语义embedding
        for i, block in enumerate(block_list):
            semantic_embeddings.append(
                self.gat_layers[i](block, h_list[i]).flatten(1)
            )
        semantic_embeddings = torch.stack(
            semantic_embeddings, dim=1
        )  # (N, M, D * K)
        print(semantic_embeddings.shape)
## 计算不同元路径的重要性从而得到最终的节点embedding表示
        return self.semantic_attention(semantic_embeddings)  # (N, D * K)


## HAN模型构建

In [20]:
class HAN(nn.Module):
    def __init__(
        self, num_metapath, in_size, hidden_size, out_size, num_heads, dropout
    ):
        super(HAN, self).__init__()

        self.layers = nn.ModuleList()
        self.layers.append(
            HANLayer(num_metapath, in_size, hidden_size, num_heads[0], dropout)
        )
## 多头注意力机制
        for l in range(1, len(num_heads)):
            self.layers.append(
                HANLayer(
                    num_metapath,
                    hidden_size * num_heads[l - 1],
                    hidden_size,
                    num_heads[l],
                    dropout,
                )
            )
        self.predict = nn.Linear(hidden_size * num_heads[-1], out_size)

    def forward(self, g, h):
        for gnn in self.layers:
            h = gnn(g, h)

        return self.predict(h)


In [21]:
model = HAN(
        num_metapath=len(metapath_list),
        in_size=features.shape[1],
        hidden_size=args.hidden_units,
        out_size=n_classes,
        num_heads=args.num_heads,
        dropout=args.dropout,
    ).to(args.device)

## 显示可训练参数

In [22]:
total_params = sum(p.numel() for p in model.parameters())
print("total_params: {:d}".format(total_params))
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("total trainable params: {:d}".format(total_trainable_params))

total_params: 252611
total trainable params: 252611


## 设定停止器

In [23]:
import datetime
class EarlyStopping(object):
    def __init__(self, patience=10):
        dt = datetime.datetime.now()
## 设定文件名代入时间
        self.filename = "early_stop_{}_{:02d}-{:02d}-{:02d}.pth".format(
            dt.date(), dt.hour, dt.minute, dt.second
        )
## 表示连续多少个epoch中验证损失没有改善之后，停止训练
        self.patience = patience
        self.counter = 0
        self.best_acc = None
        self.best_loss = None
        self.early_stop = False
        
## 每个epoch后调用的方法，用于检查是否应该停止训练
    def step(self, loss, acc, model):
        if self.best_loss is None:
            self.best_acc = acc
            self.best_loss = loss
            self.save_checkpoint(model)
## 如果当前的loss大于最佳的loss且acc小于最佳的acc，则增加计数器counter，并打印出早期停止的计数器信息。
        elif (loss > self.best_loss) and (acc < self.best_acc):
            self.counter += 1
            print(
                f"EarlyStopping counter: {self.counter} out of {self.patience}"
            )
    ##如果counter达到了设定的patience值，将early_stop标志设置为True，表示应该停止训练
            if self.counter >= self.patience:
                self.early_stop = True
## 如果当前的loss小于或等于最佳的loss且acc大于或等于最佳的acc，则更新最佳的loss和acc，并重置计数器counter为0
        else:
            if (loss <= self.best_loss) and (acc >= self.best_acc):
                self.save_checkpoint(model)
            self.best_loss = np.min((loss, self.best_loss))
            self.best_acc = np.max((acc, self.best_acc))
            self.counter = 0
        return self.early_stop
## 保存当前模型的状态字典到文件中，以便后续可以加载该模型。文件名包含了时间戳信息。
    def save_checkpoint(self, model):
        """Saves model when validation loss decreases."""
        torch.save(model.state_dict(), self.filename)
## 加载最近保存的模型的状态字典，以便恢复模型的权重和参数
    def load_checkpoint(self, model):
        """Load the latest checkpoint."""
        model.load_state_dict(torch.load(self.filename))


In [24]:
stopper = EarlyStopping(patience=args.patience)

## 损失函数设定

In [25]:
loss_fn = torch.nn.CrossEntropyLoss()

## 优化器设定

In [26]:
optimizer = torch.optim.Adam(
        model.parameters(), lr=args.lr, weight_decay=args.weight_decay
    )

## 得分函数设定

In [27]:
from sklearn.metrics import f1_score
def score(logits, labels):
##从模型的预测logits中找到每个样本的最大值及其对应的索引。
    _, indices = torch.max(logits, dim=1)
##将预测的类别索引从PyTorch张量转换为NumPy数组
    prediction = indices.long().cpu().numpy()
##将真实的标签从PyTorch张量转换为NumPy数组
    labels = labels.cpu().numpy()
## 计算分类准确率
    accuracy = (prediction == labels).sum() / len(prediction)
## 计算微平均F1分数
    micro_f1 = f1_score(labels, prediction, average="micro")
## 宏平均F1分数
    macro_f1 = f1_score(labels, prediction, average="macro")

    return accuracy, micro_f1, macro_f1

## 评估函数设定

In [28]:
def evaluate(
    model,
    g,
    metapath_list,
    num_neighbors,
    features,
    labels,
    val_nid,
    loss_fcn,
    batch_size,
):
    model.eval()

    han_valid_sampler = HANSampler(
        g, metapath_list, num_neighbors=num_neighbors * 2
    )
    dataloader = DataLoader(
        dataset=val_nid,
        batch_size=batch_size,
        collate_fn=han_valid_sampler.sample_blocks,
        shuffle=False,
        drop_last=False,
        num_workers=0,
    )
    correct = total = 0
## 这些列表用于存储每个数据批次的预测和真实标签。
    prediction_list = []
    labels_list = []
    ##设置非梯度计算
    with torch.no_grad():
        for step, (seeds, blocks) in enumerate(dataloader):
        ## 采样子图中节点的特征张量。
            h_list = load_subtensors(blocks, features)
            blocks = [block.to(args.device) for block in blocks]
            hs = [h.to(args.device) for h in h_list]
        
            logits = model(blocks, hs)
            loss = loss_fcn(
                logits, labels[numpy.asarray(seeds)].to(args.device)
            )
            # get each predict label
            _, indices = torch.max(logits, dim=1)
            prediction = indices.long().cpu().numpy()
            labels_batch = labels[numpy.asarray(seeds)].cpu().numpy()

            prediction_list.append(prediction)
            labels_list.append(labels_batch)

            correct += (prediction == labels_batch).sum()
            total += prediction.shape[0]

    total_prediction = numpy.concatenate(prediction_list)
    total_labels = numpy.concatenate(labels_list)
    micro_f1 = f1_score(total_labels, total_prediction, average="micro")
    macro_f1 = f1_score(total_labels, total_prediction, average="macro")
    accuracy = correct / total

    return loss, accuracy, micro_f1, macro_f1



## 训练过程和测试过程

In [29]:
import numpy
for epoch in range(args.num_epochs):
    model.train()
    for step, (seeds, blocks) in enumerate(dataloader):
        h_list = load_subtensors(blocks, features)
        blocks = [block.to(args.device) for block in blocks]
        hs = [h.to(args.device) for h in h_list]

        logits = model(blocks, hs)
        loss = loss_fn(
            logits, labels[numpy.asarray(seeds)].to(args.device)
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print info in each batch
        train_acc, train_micro_f1, train_macro_f1 = score(
            logits, labels[numpy.asarray(seeds)]
        )
        print(
            "Epoch {:d} | loss: {:.4f} | train_acc: {:.4f} | train_micro_f1: {:.4f} | train_macro_f1: {:.4f}".format(
                epoch + 1, loss, train_acc, train_micro_f1, train_macro_f1
            )
        )
    val_loss, val_acc, val_micro_f1, val_macro_f1 = evaluate(
        model,
        g,
        metapath_list,
        num_neighbors,
        features,
        labels,
        val_nid,
        loss_fn,
        args.batch_size,
    )
    early_stop = stopper.step(val_loss.data.item(), val_acc, model)

    print(
        "Epoch {:d} | Val loss {:.4f} | Val Accuracy {:.4f} | Val Micro f1 {:.4f} | Val Macro f1 {:.4f}".format(
            epoch + 1, val_loss.item(), val_acc, val_micro_f1, val_macro_f1
        )
    )

    if early_stop:
        break

stopper.load_checkpoint(model)
test_loss, test_acc, test_micro_f1, test_macro_f1 = evaluate(
    model,
    g,
    metapath_list,
    num_neighbors,
    features,
    labels,
    test_nid,
    loss_fn,
    args.batch_size,
)
print(
    "Test loss {:.4f} | Test Accuracy {:.4f} | Test Micro f1 {:.4f} | Test Macro f1 {:.4f}".format(
        test_loss.item(), test_acc, test_micro_f1, test_macro_f1
    )
)

torch.Size([32, 2, 64])
Epoch 1 | loss: 1.0723 | train_acc: 0.4062 | train_micro_f1: 0.4062 | train_macro_f1: 0.3571
torch.Size([32, 2, 64])
Epoch 1 | loss: 1.1265 | train_acc: 0.3438 | train_micro_f1: 0.3438 | train_macro_f1: 0.2667
torch.Size([32, 2, 64])
Epoch 1 | loss: 1.0040 | train_acc: 0.6875 | train_micro_f1: 0.6875 | train_macro_f1: 0.3526
torch.Size([32, 2, 64])
Epoch 1 | loss: 1.1134 | train_acc: 0.3750 | train_micro_f1: 0.3750 | train_macro_f1: 0.1905
torch.Size([32, 2, 64])
Epoch 1 | loss: 0.9674 | train_acc: 0.5000 | train_micro_f1: 0.5000 | train_macro_f1: 0.2222
torch.Size([32, 2, 64])
Epoch 1 | loss: 1.0071 | train_acc: 0.5000 | train_micro_f1: 0.5000 | train_macro_f1: 0.2222
torch.Size([32, 2, 64])
Epoch 1 | loss: 0.9478 | train_acc: 0.5938 | train_micro_f1: 0.5938 | train_macro_f1: 0.3401
torch.Size([32, 2, 64])
Epoch 1 | loss: 0.9524 | train_acc: 0.5938 | train_micro_f1: 0.5938 | train_macro_f1: 0.2484
torch.Size([32, 2, 64])
Epoch 1 | loss: 1.0383 | train_acc: 0.40

KeyboardInterrupt: 